## Análise de Viabilidade Econômico-Financeira por Fluxo de Caixa

#### Instalação das dependências

In [1]:
pip install pandas sqlalchemy psycopg2-binary numpy-financial

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB 325.1 kB/s eta 0:00:09
   -- ------------------------------------- 0.1/2.7 MB 1.1 MB/s eta 0:00:03
   ------------ --------------------------- 0.8/2.7 MB 4.9 MB/s eta 0:00:01
   -------------------------------------- - 2.6/2.7 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


#### Conexão com PostgreSQL

In [1]:
from sqlalchemy import create_engine
import getpass

# Parâmetros fixos do banco
DB_HOST = "srv301"
DB_PORT = 5432
DB_NAME = "geppi"

# Inputs interativos
DB_USER = input("Digite o usuário do banco: ")
DB_PASSWORD = getpass.getpass("Digite a senha do banco: ")

# Criação da engine
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Teste simples de conexão
with engine.connect() as conn:
    print("Conexão com o banco realizada com sucesso.")

Conexão com o banco realizada com sucesso.


#### Leitura da tabela do banco

In [3]:
import pandas as pd
from numpy_financial import irr, mirr

query = """
    SELECT
        id_empreendimento,
        id_setor,
        id_cenario,
        ano,
        fc
    FROM financeiro.vw_fluxo_de_caixa
    ORDER BY
        id_empreendimento,
        id_setor,
        id_cenario,
        ano
"""

df = pd.read_sql(query, engine)


#### Parâmetros financeiros

In [ ]:
taxa_financiamento = 0.1026
taxa_reinvestimento = 0.06

#Cálculo financeiro

In [6]:
# Substitui NaN e infinitos por zero
df['fc'] = (
    df['fc']
    .fillna(0)
    .replace([float('inf'), float('-inf')], 0)
)

In [7]:
# Agrupamento por projeto
grouped = df.groupby(
    ['id_empreendimento', 'id_setor', 'id_cenario']
)

resultados = []

for (id_emp, id_set, id_cen), data in grouped:
    # Série temporal do fluxo de caixa
    fluxos = data['fc'].tolist()

    # Cálculo da TIR
    irr_value = irr(fluxos)

    # Cálculo da TIRM
    mirr_value = mirr(
        fluxos,
        taxa_financiamento,
        taxa_reinvestimento
    )

    # Soma total do fluxo de caixa
    total_fc = sum(fluxos)

    resultados.append({
        'id_empreendimento': id_emp,
        'id_setor': id_set,
        'id_cenario': id_cen,
        'tir': irr_value,
        'tirm': mirr_value,
        'fluxo_caixa_total': total_fc
    })

# DataFrame final
resultado_df = pd.DataFrame(resultados)


# Separação de projetos com / sem TIR

In [9]:
projetos_com_irr = resultado_df[resultado_df['tir'].notna()]
projetos_sem_irr = resultado_df[resultado_df['tir'].isna()]

In [10]:
print(resultado_df.columns)

Index(['id_empreendimento', 'id_setor', 'id_cenario', 'tir', 'tirm',
       'fluxo_caixa_total'],
      dtype='object')


In [11]:
print(resultado_df.head(10))

   id_empreendimento  id_setor  id_cenario       tir      tirm  \
0                 11       1.0         4.0 -0.226173 -0.102620   
1                 12       1.0         4.0       NaN       NaN   
2                 13       1.0         4.0 -0.537568 -0.127070   
3                 18       1.0         1.0 -0.285765 -0.149726   
4                 18       1.0         2.0 -0.179424 -0.140917   
5                 18       1.0         3.0 -0.178771 -0.140740   
6                 18       1.0         4.0 -0.057436 -0.017193   
7                 29       1.0         2.0 -0.696189 -0.163523   
8                 29       1.0         3.0 -0.701786 -0.164068   
9                 29       1.0         4.0 -0.528885 -0.150217   

   fluxo_caixa_total  
0      -7.500577e+08  
1      -1.506578e+09  
2      -3.243097e+09  
3      -3.310845e+09  
4      -3.200099e+09  
5      -3.198789e+09  
6      -2.039792e+09  
7      -4.661398e+09  
8      -4.666063e+09  
9      -4.516998e+09  
